<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/solcast_h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas

Data available in 10 minutes only, So, we aggregate to 1 hour.

In [8]:
#@title 🌤️ 2024 Full Year Data: GHI, Temp, & Wind Speed
import requests
import pandas as pd
import time

# API Key
SOLCAST_API_KEY = "C0EcOTQBX-lI_w9Pvi_NXekq1ZLDv9No"

cities = {
    "Turku": {"lat": 60.5, "lon": 22.3},
    "Copenhagen": {"lat": 55.7, "lon": 12.6},
    "Stockholm": {"lat": 59.3, "lon": 18.1},
    "Oslo": {"lat": 59.9, "lon": 10.7}
}

months = [f"2024-{m:02d}-01" for m in range(1, 13)]
url = "https://api.solcast.com.au/data/historic/radiation_and_weather"

for city, coords in cities.items():

    all_data = []
    print(f"\n🌍 Processing 2024 for: {city}")

    for start_date in months:

        params = {
            "latitude": coords["lat"],
            "longitude": coords["lon"],
            "api_key": SOLCAST_API_KEY,
            "start": f"{start_date}T00:00:00Z",
            "duration": "P1M",          # Exactly one month
            "period": "PT10M",          # ✅ 10-minute data
            "format": "json",
            "output_parameters": "ghi,air_temp,wind_speed_10m"
        }

        try:
            res = requests.get(url, params=params)
            if res.status_code == 200:
                month_df = pd.DataFrame(res.json()['estimated_actuals'])
                all_data.append(month_df)
                print(f"  ✅ {start_date} Loaded", end="\r")
            else:
                print(f"  ❌ {start_date} Failed: {res.status_code}")
        except Exception as e:
            print(f"  ⚠️ Error: {e}")

        time.sleep(1.5)

    if all_data:

        # Combine all months
        df = pd.concat(all_data).drop_duplicates(subset=['period_end'])
        df['period_end'] = pd.to_datetime(df['period_end'])

        # Rename columns
        rename_map = {
            'period_end': 'Timestamp_UTC',
            'ghi': 'GHI_W_m2',
            'air_temp': 'Air_Temp_C',
            'wind_speed_10m': 'Wind_Speed_m_s'
        }

        final_df = df[[col for col in rename_map if col in df.columns]].rename(columns=rename_map)
        final_df = final_df.sort_values('Timestamp_UTC')

        # ===============================
        # SAVE 10-MINUTE DATA
        # ===============================
        file_10min = f"{city}_Weather_Solar_2024_10min.csv"
        final_df.to_csv(file_10min, index=False)

        print(f"\n💾 10-min file saved: {file_10min}")
        print(f"   Rows: {len(final_df)}")

        # ===============================
        # CREATE HOURLY DATA
        # ===============================
        hourly_df = final_df.copy()
        hourly_df = hourly_df.set_index("Timestamp_UTC")

        # Mean aggregation to hourly
        hourly_df = hourly_df.resample("1H").mean()

        hourly_df = hourly_df.reset_index()

        # ===============================
        # SAVE HOURLY DATA
        # ===============================
        file_hourly = f"{city}_Weather_Solar_2024_hourly.csv"
        hourly_df.to_csv(file_hourly, index=False)

        print(f"💾 Hourly file saved: {file_hourly}")
        print(f"   Rows: {len(hourly_df)}")

print("\n🚀 All cities completed!")


🌍 Processing 2024 for: Turku
  ❌ 2024-01-01 Failed: 403
  ❌ 2024-02-01 Failed: 403


KeyboardInterrupt: 